In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10,6)
color = sns.color_palette()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [40]:
# customize the display area of the dataset
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [41]:
df = pd.read_csv('Train.csv')

In [42]:
df.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [43]:
df.shape

(23524, 13)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [45]:
df.isnull().sum()

country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64

From the above analysis, there's no missing value in the dataset

In [46]:
for i in df.columns:
    print(i, df[i].unique())
    print("**********************")
    print("***********************")

country ['Kenya' 'Rwanda' 'Tanzania' 'Uganda']
**********************
***********************
year [2018 2016 2017]
**********************
***********************
uniqueid ['uniqueid_1' 'uniqueid_2' 'uniqueid_3' ... 'uniqueid_8757'
 'uniqueid_8758' 'uniqueid_8759']
**********************
***********************
bank_account ['Yes' 'No']
**********************
***********************
location_type ['Rural' 'Urban']
**********************
***********************
cellphone_access ['Yes' 'No']
**********************
***********************
household_size [ 3  5  8  7  1  6  4 10  2 11  9 12 16 15 13 14 21 18 17 20]
**********************
***********************
age_of_respondent [ 24  70  26  34  32  42  54  76  40  69  64  31  38  47  27  48  25  21
  18  22  58  55  62  29  35  45  67  19  80  66  50  33  28  51  16  17
  30  37  59  65  46  56  52  23  43  49  44  72  53  63  39  81  78  36
  20  60  95  71  57  85  68  41  61  75  86  73  93  74  88  90  77  84
  82  89  79  83  94  87

In [47]:
df.nunique()

country                      4
year                         3
uniqueid                  8735
bank_account                 2
location_type                2
cellphone_access             2
household_size              20
age_of_respondent           85
gender_of_respondent         2
relationship_with_head       6
marital_status               5
education_level              6
job_type                    10
dtype: int64

#### Convert all binary data into 1s and 0s.

In [48]:
def bank_account_num(x):
    if x == "Yes":
        return 1
    else:
        return 0


def gender_num(x):
    if x == "Male":
        return 1
    else:
        return 0

def location_num(x):
    if x == "Urban":
        return 1
    return 0

def phone_num(x):
    if x == "Yes":
        return 1
    return 0


#### Convert the binary data into 1s and 0s

In [49]:
df['bank_account'] = df['bank_account'].apply(lambda x: bank_account_num())
df['cellphone_access'] = df['cellphone_access'].apply(lambda x: phone_num(x))
df['location_type'] = df['location_type'].apply(lambda x: location_num(x))
df['gender_of_respondent'] = df['gender_of_respondent'].apply(lambda x: gender_num(x))


TypeError: bank_account_num() missing 1 required positional argument: 'x'

In [ ]:
df.nunique()

country                      4
year                         3
uniqueid                  8735
bank_account                 1
location_type                1
cellphone_access             1
household_size              20
age_of_respondent           85
gender_of_respondent         1
relationship_with_head       6
marital_status               5
education_level              6
job_type                    10
dtype: int64